# Written Exercise 6.

Train 2 MNIST models from different sub-sections of the data. Attack one of the MNIST models and see performance of attack on each of the models. The goal is to see whether attacks are transferable between different models.

In [27]:
%matplotlib agg
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from tqdm.notebook import tqdm

from IPython.display import clear_output

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import gridspec

import hw4_utils
import hw4_mnist
import hw4_part1

### import two instances of the MNIST model and reset their parameters.

In [23]:
# Bring model inand reset
m1 = hw4_mnist.MNISTModel()
m1.build()

m2 = hw4_mnist.MNISTModel()
m2.build()

### Import MNIST data split to - train 1, train 2, test

In [24]:
mnist_data = hw4_mnist.load_data()

cut_idx = 25000

train1_X = mnist_data.train.X[0:cut_idx]
train1_Y = mnist_data.train.Y[0:cut_idx]

train2_X = mnist_data.train.X[cut_idx:]
train2_Y = mnist_data.train.Y[cut_idx:]

test_X = mnist_data.test.X
test_Y = mnist_data.test.Y

### Train the 2 models on different test data

In [25]:
m1.train(train1_X, train1_Y, epochs = 1, batch_size = 16)
m2.train(train2_X, train2_Y, epochs = 1, batch_size = 16)

Train on 25000 samples
25000/25000 [==============================] - 7s 290us/sample - loss: 0.2811 - acc: 0.9143
Train on 25000 samples
25000/25000 [==============================] - 7s 289us/sample - loss: 0.2956 - acc: 0.9080


### Attack the first model and obtain adversarial test data

In [31]:
# PGD Clip on M1
att1 = hw4_part1.PGDAttacker(
        model=m1,
        num_steps = 10,
        target=5,
        learning_rate_decay=0.9,
        learning_rate=1.0,
        step_mode="clip",
        c=0.2
    )

test1 = att1.attack(test_X,test_Y)

attacking:  10%|▉         | 61/625 [00:11<01:36,  5.83batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  51%|█████     | 318/625 [00:55<00:57,  5.35batch/s]


attacking:  61%|██████    | 382/625 [01:07<00:40,  6.07batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  81%|████████  | 507/625 [01:31<00:26,  4.46batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


In [32]:
# PGD Project on M1
att2 = hw4_part1.PGDAttacker(
        model=m1,
        num_steps = 10,
        target=5,
        learning_rate_decay=0.9,
        learning_rate=1.0,
        step_mode="project",
        c=0.2
    )

test2 = att2.attack(test_X,test_Y)

attacking:  10%|█         | 65/625 [00:14<01:52,  4.96batch/s]


attacking:  21%|██        | 130/625 [00:27<01:43,  4.80batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  41%|████▏     | 259/625 [00:54<01:17,  4.73batch/s]


attacking batch:  90%|█████████ | 9/10 [00:00<00:00, 88.28step/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  72%|███████▏  | 448/625 [01:36<00:42,  4.12batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  92%|█████████▏| 577/625 [02:05<00:10,  4.66batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


In [33]:
# CWL2 on M1
att3 = att = hw4_part1.CWL2Attacker(
        model=m1,
        num_steps=10,
        target=5,
        learning_rate=1.0,
        learning_rate_decay=0.9,
        k=0,
        c=2
    )

test3 = att3.attack(test_X,test_Y)

attacking:  10%|█         | 65/625 [00:13<01:45,  5.29batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  31%|███▏      | 196/625 [00:38<01:24,  5.10batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  52%|█████▏    | 327/625 [01:04<01:01,  4.85batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  73%|███████▎  | 458/625 [01:33<00:37,  4.45batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking:  94%|█████████▍| 589/625 [02:04<00:08,  4.25batch/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


### Analyze test accuracy on models m1,m2 given each of the attack types.

In [41]:
loss_m1_att1 = m1.model.evaluate(test1,test_Y)
loss_m1_att2 = m1.model.evaluate(test2,test_Y)
loss_m1_att3 = m1.model.evaluate(test3,test_Y)

loss_m2_att1 = m2.model.evaluate(test1,test_Y)
loss_m2_att2 = m2.model.evaluate(test2,test_Y)
loss_m2_att3 = m2.model.evaluate(test3,test_Y)

10000/10000 [==============================] - 1s 70us/sample - loss: 0.4781 - acc: 0.8115


In [42]:
loss_m1_og = m1.model.evaluate(test_X,test_Y)

loss_m2_og = m2.model.evaluate(test_X,test_Y)

10000/10000 [==============================] - 1s 63us/sample - loss: 0.1169 - acc: 0.9648
